# All imports and inits

In [1]:
from dotenv import load_dotenv
from transformers import AutoModel, AutoTokenizer
from groq import Groq
from pinecone import Pinecone

import os
import gradio as gr
import time

# Important: Import pinecone-client properly
# Load environment variables from .env file
load_dotenv()

DATA_PATH = os.getenv("DATA_PATH")
PINECONE_API = os.getenv("PINECONE_API")
PINECONE_ENV = os.getenv("PINECONE_ENV")

GROQ_API_KEY = os.getenv("GROQ_API_KEY")
GROQ_CHAT_URL = "https://api.groq.com/openai/v1/chat/completions"
# Configure headers for Groq API requests
GROQ_HEADERS = {
    "Authorization": f"Bearer {GROQ_API_KEY}",
    "Content-Type": "application/json",
}

LLM_MODEL = "llama-3.3-70b-versatile"


def track_time(func):
    def wrapper(*args, **kwargs):
        start = time.perf_counter()
        result = func(*args, **kwargs)
        end = time.perf_counter()
        print(f"[Time Tracker] `{func.__name__}` took {end - start:.4f} seconds")
        return result

    return wrapper


d:\Disrupt Labs\RAG-workshop-Bahria-University\env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


# Init Pinecone

In [2]:
pc = Pinecone(api_key=PINECONE_API)
#print(PINECONE_API)


# Embedding Function



In [3]:
# Connect to the index
index = pc.Index("bahria")
# index = pc.Index("ahsan-400pg-pdf-doc-test")

embedding_model = AutoModel.from_pretrained(
    "jinaai/jina-embeddings-v2-base-en", trust_remote_code=True
)


# Function to generate embeddings without tokenization
@track_time
def get_embedding(data):
    embeddings = embedding_model.encode(data).tolist()
    return embeddings


ImportError: 
AutoModel requires the PyTorch library but it was not found in your environment. Checkout the instructions on the
installation page: https://pytorch.org/get-started/locally/ and follow the ones that match your environment.
Please note that you may need to restart your runtime after installation.


# Query Pinecone


In [6]:
# Function to query Pinecone index using embeddings
@track_time
def query_pinecone(embedding):
    # Use keyword arguments to pass the embedding and other parameters
    result = index.query(vector=embedding, top_k=20, include_metadata=True)
    return result["matches"]


print(query_pinecone(get_embedding("Pediatric surgery definition")))


[Time Tracker] `get_embedding` took 0.1333 seconds
[Time Tracker] `query_pinecone` took 1.7823 seconds
[{'id': 'vec_36',
 'metadata': {'chunk_id': 0.0,
              'file_type': 'excel',
              'source': 'D:\\Disrupt '
                        'Labs\\RAG-workshop-Bahria-University\\data\\Exam '
                        '01.xlsx',
              'text': 'Bank Name: nan Question: Also called an Outpatient '
                      'Center, a(n) __________  is a self-contained or '
                      'freestanding facility that focuses on "day-surgery" '
                      'meaning patients will go home after their procedures. '
                      'Randomization (Yes/No): Yes Correct Answer Number: 1 '
                      'Answer 1: Ambulatory Surgery Facility Feedback 1: nan '
                      'Answer 2: Sterile Processing Department Feedback 2: nan '
                      'Answer 3: Nuclear Medicine Department Feedback 3: nan '
                      'Answer 4: Trauma 

# Query Groq Inference

| Use Case | Recommended top_p | Notes |
|----------|------------------|-------|
| Factual Q&A | 0.1 - 0.3 | Lower values for more deterministic, factual responses |
| Code Generation | 0.2 - 0.5 | Precision matters more than creativity |
| Technical Writing | 0.5 - 0.7 | Balanced approach for technical accuracy with clarity |
| General Conversation | 0.7 - 0.9 | Good balance for most chatbot applications |
| Creative Writing | 0.9 - 1.0 | Higher values for more diverse and creative outputs |

\n
| Parameter Combination | Use Case |
|----------------------|----------|
| top_p=0.5, temperature=0.3 | Highly factual, consistent responses |
| top_p=0.7, temperature=0.5 | Educational content with examples |
| top_p=0.9, temperature=0.7 | Creative but coherent responses |

In [ ]:
@track_time
def query_groq(user_prompt, sys_prompt):
    client = Groq(api_key=os.environ["GROQ_API_KEY"])

    # Always use streaming mode
    return client.chat.completions.create(
        model=LLM_MODEL,  # or whichever model you're using
        temperature=0.5,
        messages=[
            {"role": "system", "content": sys_prompt},
            {"role": "user", "content": user_prompt},
        ],
        stream=True,
        # top_p=0.7,  # testing for better results
    )


# Print all tool calls
# print(completion.choices[0].message.executed_tools)

# Tokenizer to count number of tokens
tokenizer = AutoTokenizer.from_pretrained("jinaai/jina-embeddings-v2-base-en")


@track_time
def count_tokens(text: str) -> int:
    # Encode the text into tokens
    tokens = tokenizer.encode(text)
    return len(tokens)


# Full SYSTEM MESSAGE
    '''- Provide clear, step-by-step explanations to ensure deep understanding.
    - Use chain-of-thought reasoning to thoroughly evaluate the provided context before responding.
    - Ask guiding questions to encourage critical thinking.
    - Adapt your explanation to match the student's knowledge level.
    - Strictly use terminologies provided in the given context.
    - Provide short, ideal examples (2–3) to illustrate your points clearly.
    - Blend learning material with your own knowledge while ensuring answers stay within the healthcare context.
    - **Always provide all specific relevant sources with name from the context in your responses: URLs, video names, video timestamps, links, resources, *ebook names*, lesson names, lesson numbers. If the user query is not relevant to the context, do not provide any references and sources.**
    - Perform sentiment analysis based on conversation history and user queries to adapt your responses empathetically and effectively.
    - Must provide all relevant video timestamps with video name (start and end). If timestamps aren't available, tell the user to watch the whole video.
    - Provide a thoughtful and contextually accurate response.
    - No repetition in responses.
    - **If student asks something completely out of context, politely decline and ask them to ask a question related to their course. Do not provide any references or sources.**
    - Please avoid using "Bank Name".
    - When users request questions, answers, quizzes, or exams, generate high-quality educational assessments directly from the learning material. Use various question types (e.g., multiple-choice with 4–5 options, true/false, short answer, fill-in-the-blanks, essay), clear formatting, different cognitive levels (recall, comprehension, application, analysis), detailed answer keys with explanations, consistent formatting (numbered questions, lettered options, bold correct answers), difficulty adaptation, and step-by-step solutions.
    - Never generate fabricated information when providing references or sources. Only use facts, references, citations, lesson names, e-book titles, video names, and timestamps explicitly present in the provided learning materials.
    - Always provide all references and sources **only at the end of the response.**. Do **not** include fill-in-the-blanks, quizzes, or Q&A content in the references and sources section. Make sure you mention what type of reference / source it is.
    - Incorporate occasional follow-up questions or prompts (e.g., “Would you like to see an example?” or “Need a quick quiz to check your understanding?”) to promote engagement and deeper learning.
    - Default to concise, scannable answers. Use bullet points and bolding. Only give longer explaination and details if the user requests it or implies it.
    - Donot provide references or sources between text or at the end of a sentence, only under **References and Sources**.
    - For followup questions and queries , use previous conversation history and context to answer the user query.
    - Be more confident and assertive in your responses.
    - ** If a user asks about anything related to certification, licensing, externships, or career pathways, provide clear, specific information. This includes queries that mention or imply:

        “certification”
        “certificate”
        “get certified”
        “license” / “licensing”
        “externship”
        “internship”
        “official requirement”
        “exam eligibility”
        “career path”
        “qualification”
        “accreditation”
        “approved training”
        “enrollment”
        “program approval”

        if such keywords are present then **always ask user to contact official support for confirmation or further help,** while including the contact details below:

        Support Contact Information:
        📞 Phone: 1-800-555-HEAL (4325)
        📧 Email: support@healthylifehelp.org
        🌐 Website: www.healthylifehelp.org
        🏥 Address: 123 Wellness Blvd, Caretown, CA 90210
        🕒 Hours: Mon–Fri, 8 AM – 6 PM (PST) '''

# Process User Query

### Gradio GUI TEST

In [ ]:
# --------------------------------------------------------- ## Groq and Gradio with Streaming Enabled -----------------------------------------------------
# Modified process_user_query to properly yield streaming updates
@track_time
def process_user_query(user_query: str, conversation_history: list):
    print(f"User Query Tokens: {count_tokens(user_query)}")

    # Generate embedding and get relevant context
    embedding = get_embedding(user_query)
    relevant_chunks = query_pinecone(embedding)
    context = "\n".join(chunk["metadata"]["text"] for chunk in relevant_chunks)

    # Format conversation history for the prompt
    history_str = "\n".join(
        f"User: {user}\nCoach: {response}" for user, response in conversation_history
    )

    # --------------------------------------------- UPDATED SYSTEM AND USER PROMPT ------------------------------------------------------------------------
    # System prompt - contains instructions for the AI's behavior
    system_prompt = f"""
    
    Conversation history:
    {history_str}
    
    learning materials:
    {context}
    
    You are an expert, knowledgeable, and friendly coach. Follow these **guidelines** carefully:

   
    1. Provide clear, step-by-step explanations to ensure deep understanding.
    2. Use chain-of-thought reasoning to thoroughly evaluate the provided context before responding.
    3. Ask guiding questions to encourage critical thinking.
    4. Adapt your explanation to match the student's knowledge level.
    5. Strictly use terminologies provided in the given context.
    6. Provide short, ideal examples (2-3) to illustrate your points clearly.
    7. Only answer based on the provided context—do not speculate or include external information.
    8. Explicitly cite the sources from the context in your responses.
    9. Perform sentiment analysis based on conversation history and user queries to adapt your responses empathetically and effectively  but don't mention it in the response.

    """

    # User prompt - contains the specific query and context
    user_prompt = f"""
    
    
    New student question:
    "{user_query}"

    """

    # Then in your query_groq function:
    stream_response = query_groq(user_prompt, system_prompt)

    # The function now directly yields the stream chunks for the Gradio interface to use
    full_response = ""

    # First, yield a response with empty text to set up the message
    # This creates the user message immediately
    temp_history = conversation_history.copy()
    temp_history.append((user_query, ""))
    yield temp_history, context

    # Process the stream
    for chunk in stream_response:
        if (
            hasattr(chunk.choices[0].delta, "content")
            and chunk.choices[0].delta.content is not None
        ):
            content_chunk = chunk.choices[0].delta.content
            full_response += content_chunk

            # Create a temporary history with the current response
            temp_history = conversation_history.copy()
            temp_history.append((user_query, full_response))

            # Yield the updated history for display
            yield temp_history, context

    # Return the final history with the complete response
    final_history = conversation_history.copy()
    final_history.append((user_query, full_response))
    yield final_history, context


@track_time
def create_gradio_interface(conversation_history):
    with gr.Blocks() as interface:
        gr.Markdown("# 🧑‍🏫 AI Coaching Assistant")
        gr.Markdown("Welcome! I'm here to help you learn. Type your question below.")

        # State management
        chat_history = gr.State(conversation_history)

        with gr.Row():
            chatbot = gr.Chatbot(height=500)
            with gr.Column(scale=0.5):
                context_display = gr.Textbox(
                    label="Relevant Context", interactive=False
                )

        user_input = gr.Textbox(label="Your Question", placeholder="Type here...")

        with gr.Row():
            submit_btn = gr.Button("Submit", variant="primary")
            undo_btn = gr.Button("Undo Last")
            clear_btn = gr.Button("Clear History")

        def handle_submit(user_query, history):
            if not user_query.strip():
                return gr.update(), history, ""

            # Use the generator directly from process_user_query
            # This will yield incremental updates as they arrive
            response_generator = process_user_query(user_query, history)

            for updated_history, context in response_generator:
                # Directly update the chatbot with each streaming chunk
                yield "", updated_history, context, updated_history

        # Component interactions with streaming support
        submit_btn.click(
            handle_submit,
            [user_input, chat_history],
            [user_input, chat_history, context_display, chatbot],
        )

        # Add submit on Enter key press
        user_input.submit(
            handle_submit,
            [user_input, chat_history],
            [user_input, chat_history, context_display, chatbot],
        )

        undo_btn.click(
            lambda history: history[:-1] if history else [],
            [chat_history],
            [chat_history],
        ).then(lambda x: x, [chat_history], [chatbot])

        clear_btn.click(lambda: [], None, [chat_history]).then(
            lambda: ([], ""), None, [chatbot, context_display]
        )

    return interface


def main():
    """
    Main entry point for the application.

    Initializes the conversation history with a welcome message,
    creates the Gradio interface, and launches the web app.
    """
    # Initialize conversation history with welcome message
    welcome_message = "Hi there! I'm your AI coach. I can help answer questions about your course materials, explain difficult concepts, and guide your learning journey. What would you like to know today?"
    initial_conversation_history = [("", welcome_message)]

    # Create and launch the interface
    interface = create_gradio_interface(initial_conversation_history)
    interface.launch(share=True)


if __name__ == "__main__":
    main()


C:\Users\Osama\AppData\Local\Temp\ipykernel_31620\1872999044.py:95: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot(height=500)
d:\Disrupt Labs\RAG-workshop-Bahria-University\bahria-env\Lib\site-packages\gradio\layouts\column.py:59: UserWarning: 'scale' value should be an integer. Using 0.5 will cause issues.
  warnings.warn(


[Time Tracker] `create_gradio_interface` took 0.1098 seconds
* Running on local URL:  http://127.0.0.1:7861
* Running on public URL: https://3dac64712cf31c4f38.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


[Time Tracker] `process_user_query` took 0.0000 seconds
[Time Tracker] `count_tokens` took 0.0002 seconds
User Query Tokens: 6
[Time Tracker] `query_pinecone` took 1.3900 seconds
[Time Tracker] `query_groq` took 1.0144 seconds
[Time Tracker] `process_user_query` took 0.0000 seconds
[Time Tracker] `count_tokens` took 0.0002 seconds
User Query Tokens: 11
[Time Tracker] `query_pinecone` took 0.2159 seconds
[Time Tracker] `query_groq` took 0.9075 seconds
